## NOMS Paper

In [ ]:
using DataFrames
using JSON
using ParsimoniousMonitoring: LogEntry
using Statistics

In [ ]:
parse_log(d::Dict) = LogEntry(map(x -> d[string(x)], fieldnames(LogEntry))...)

function load_results(filename)
    obj = JSON.parsefile(filename)
    Dict(k => map(parse_log, v) for (k, v) in obj)
end

### 5.2 Validation against real latency data

In [ ]:
scenarios = [
    "../results/Haifa_Santiago.json",
    "../results/Paris_Santiago.json",
    "../results/Paris_Tokyo.json",
    "../results/Singapore_HongKong.json"
]

function min_delay_gap(log)
    delay = sum(skipmissing(map(x -> x.delay, log)))
    delay_opt = sum(skipmissing(map(x -> x.delay_opt, log)))
    (delay - delay_opt) / delay_opt
end

metrics = Dict(
    "processing_time" => log -> mean(x -> x.time, log),
    "avg_measures" => log -> mean(x -> sum(x.a), log),
    "min_delay_reached" => log -> mean(x -> x.delay == x.delay_opt, log),
#     "min_delay_gap" => log -> min_delay_gap(log)
);

In [ ]:
df = DataFrame([String[], String[], String[], Real[]], [:scenario, :policy, :metric, :value])
for scenario in scenarios
    name = splitext(basename(scenario))[1]
    results = load_results(scenario)    
    for (policy, log) in results, (metric, f) in metrics
        push!(df, (name, policy, metric, f(log)))
    end
end
df = unstack(df, :metric, :value);

In [ ]:
df[df.policy .== "Heuristic",:]